Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [ ]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [ ]:
CONSTRAINTS

In [ ]:
def fitness(solution):
    return np.sum(VALUES[solution >= 0])

def is_feasible(solution):
    for k in range(NUM_KNAPSACKS):
        if np.any(np.sum(WEIGHTS[solution == k], axis=0) > CONSTRAINTS[k]):
            return False
    return True

In [ ]:
solution = np.random.randint(-1, NUM_KNAPSACKS, size=NUM_ITEMS)
solution = np.ones(NUM_ITEMS, dtype=int) * -1

In [ ]:
def tweak(solution):
    i = np.random.randint(0, NUM_ITEMS)
    feasible_choices = np.arange(-1, NUM_KNAPSACKS)
    probabilities = np.array([0.5] + [0.5 / NUM_KNAPSACKS] * NUM_KNAPSACKS)
    probabilities = None
    new_value = np.random.choice(feasible_choices, p=probabilities)
    while new_value == solution[i]:
        new_value = np.random.choice(feasible_choices, p=probabilities)
    new_solution = solution.copy()
    new_solution[i] = new_value
    if new_value == -1:
        old_value = new_solution[i]
        unassigned_items = np.arange(NUM_ITEMS)[solution == -1]
        if len(unassigned_items) >= 1:
            j = np.random.choice(unassigned_items)
            new_solution[j] = old_value
    return new_solution

In [ ]:
def hc_solver(solution, max_iter=10000):
    best_solution = solution.copy()
    best_fitness = fitness(solution) if is_feasible(solution) else 0
    fitness_history = [best_fitness.item()]
    for _ in range(max_iter):
        new_solution = tweak(best_solution)
        if is_feasible(new_solution):
            new_fitness = fitness(new_solution)
            if new_fitness > best_fitness:
                best_solution = new_solution
                best_fitness = new_fitness
        fitness_history.append(best_fitness.item())
    return best_solution, best_fitness, fitness_history

In [ ]:
def hc_steepest_ascent_solver(solution, max_iter=1000):
    best_solution = solution.copy()
    best_fitness = fitness(solution) if is_feasible(solution) else 0
    fitness_history = [best_fitness.item()]
    for _ in range(max_iter):
        neighbors = [tweak(best_solution) for _ in range(10)]
        feasible_neighbors = [s for s in neighbors if is_feasible(s)]
        if not feasible_neighbors:
            continue
        new_solution = max(feasible_neighbors, key=fitness)
        new_fitness = fitness(new_solution)
        if new_fitness > best_fitness:
            best_solution = new_solution
            best_fitness = new_fitness
        fitness_history.append(best_fitness.item())
    return best_solution, best_fitness, fitness_history

In [ ]:
def sa_solver(solution, max_iter=10000, initial_temp=100, cooling_rate=0.99):
    current_solution = solution.copy()
    current_fitness = fitness(solution) if is_feasible(solution) else 0
    best_solution = current_solution.copy()
    best_fitness = current_fitness
    fitness_history = [best_fitness.item()]
    temp = initial_temp
    for _ in range(max_iter):
        new_solution = tweak(current_solution)
        if is_feasible(new_solution):
            new_fitness = fitness(new_solution)
            if new_fitness > current_fitness:
                current_solution = new_solution
                current_fitness = new_fitness
                if new_fitness > best_fitness:
                    best_solution = new_solution
                    best_fitness = new_fitness
            else:
                acceptance_prob = np.exp((new_fitness - current_fitness) / temp)
                if np.random.rand() < acceptance_prob:
                    current_solution = new_solution
                    current_fitness = new_fitness
        temp *= cooling_rate
        fitness_history.append(best_fitness.item())
    return best_solution, best_fitness, fitness_history

In [ ]:
best_solution, best_fitness, fitness_history = hc_solver(solution)

plt.plot(fitness_history)
best_solution, best_fitness

In [ ]:
best_solution, best_fitness, fitness_history = hc_steepest_ascent_solver(solution)

plt.plot(fitness_history)
best_solution, best_fitness

In [ ]:
best_solution, best_fitness, fitness_history = sa_solver(solution)

plt.plot(fitness_history)
best_solution, best_fitness

## TEST PROBLEMS

In [ ]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [ ]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [ ]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)